In [3]:
import numpy as np
import os
#import poly_yolo as yolo #or "import poly_yolo_lite as yolo" for the lite version
import poly_yolo_lite as yolo

import matplotlib.pyplot as plt
import tensorflow as tf
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import keras
from keras.models import model_from_json, load_model

In [4]:
#Set the learning phase to Test since the model is already trained.
keras.backend.set_learning_phase(0)

#load pretrained model
#if you want to detect more objects, lower the score and vice versa
trained_model = yolo.YOLO(model_path='trained_model/poly_yolo_lite.h5', iou=0.5, score=0.3)

trained_model/poly_yolo_lite.h5 model, anchors, and classes loaded.


In [5]:
print(trained_model.yolo_model.layers[0].input)
print(trained_model.yolo_model.layers[-1].output)

trained_model.yolo_model.summary()

Tensor("input_1:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("conv2d_68/BiasAdd:0", shape=(?, ?, ?, 720), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 1 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 1 0           batc

In [6]:
#save checkpoint
saver = tf.train.Saver()
tf_session = keras.backend.get_session()
input_graph_def = tf_session.graph.as_graph_def()
save_path = saver.save(tf_session, './gen/checkpoint.ckpt')
tf.train.write_graph(input_graph_def, './gen', 'poly_yolo_dpu.pb', as_text=False)

'./gen/poly_yolo_dpu.pb'

In [7]:
#check the node names 
nodes_names = [node.name for node in 
               tf.get_default_graph().as_graph_def().node]
print(nodes_names)

['input_1', 'conv2d_1/random_uniform/shape', 'conv2d_1/random_uniform/min', 'conv2d_1/random_uniform/max', 'conv2d_1/random_uniform/RandomUniform', 'conv2d_1/random_uniform/sub', 'conv2d_1/random_uniform/mul', 'conv2d_1/random_uniform', 'conv2d_1/kernel', 'conv2d_1/kernel/Assign', 'conv2d_1/kernel/read', 'conv2d_1/weight_regularizer/Square', 'conv2d_1/weight_regularizer/Const', 'conv2d_1/weight_regularizer/Sum', 'conv2d_1/weight_regularizer/mul/x', 'conv2d_1/weight_regularizer/mul', 'conv2d_1/weight_regularizer/add/x', 'conv2d_1/weight_regularizer/add', 'conv2d_1/convolution/dilation_rate', 'conv2d_1/convolution', 'batch_normalization_1/Const', 'batch_normalization_1/gamma', 'batch_normalization_1/gamma/Assign', 'batch_normalization_1/gamma/read', 'batch_normalization_1/Const_1', 'batch_normalization_1/beta', 'batch_normalization_1/beta/Assign', 'batch_normalization_1/beta/read', 'batch_normalization_1/Const_2', 'batch_normalization_1/moving_mean', 'batch_normalization_1/moving_mean/As

In [8]:
!freeze_graph --help

usage: freeze_graph [-h] [--input_graph INPUT_GRAPH]
                    [--input_saver INPUT_SAVER]
                    [--input_checkpoint INPUT_CHECKPOINT]
                    [--checkpoint_version CHECKPOINT_VERSION]
                    [--output_graph OUTPUT_GRAPH]
                    [--input_binary [INPUT_BINARY]]
                    [--output_node_names OUTPUT_NODE_NAMES]
                    [--restore_op_name RESTORE_OP_NAME]
                    [--filename_tensor_name FILENAME_TENSOR_NAME]
                    [--clear_devices [CLEAR_DEVICES]]
                    [--initializer_nodes INITIALIZER_NODES]
                    [--variable_names_whitelist VARIABLE_NAMES_WHITELIST]
                    [--variable_names_blacklist VARIABLE_NAMES_BLACKLIST]
                    [--input_meta_graph INPUT_META_GRAPH]
                    [--input_saved_model_dir INPUT_SAVED_MODEL_DIR]
                    [--saved_model_tags SAVED_MODEL_TAGS]

optional arguments:
  -h, --help            show

In [9]:
#Freeze Tensorflow graph: lastnode: concat_9; lastlayer: conv2d_69/BiasAdd; lastlayer_lite: conv2d_68/BiasAdd
!freeze_graph \
    --input_meta_graph ./gen/checkpoint.ckpt.meta \
    --input_checkpoint ./gen/checkpoint.ckpt \
    --output_graph ./gen/frozen.pb \
    --output_node_names conv2d_68/BiasAdd \
    --input_binary true

Instructions for updating:
Use tf.gfile.GFile.
Loaded meta graph file './gen/checkpoint.ckpt.meta


In [10]:
#confirm available input and output node names
!vai_q_tensorflow inspect --input_frozen_graph=./gen/frozen.pb

Op types used: 417 Const, 337 Identity, 69 Mul, 68 Conv2D, 67 FusedBatchNorm, 67 Maximum, 29 Add, 5 Pad, 2 ResizeBilinear, 2 Shape, 2 StridedSlice, 1 BiasAdd, 1 Placeholder

Found 1 possible inputs: (name=input_1, type=float(1), shape=[?,?,?,3]) 
Found 1 possible outputs: (name=conv2d_68/BiasAdd, op=BiasAdd) 


#Prepare calibration data and function

In [15]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

dir_imgs_name = './seg_test_images'
list_of_imgs = [root+"/"+name for root, dirs, files in os.walk(dir_imgs_name) for name in files]    
list_of_imgs.sort()
num_imgs = 1

# get the 1st image with right shape
img = load_img(list_of_imgs[0])  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (1024, 2048, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 1024, 2048, 3)
dataset_calib = x
# fill in the following images
imgs          = 0
for im in range (1, num_imgs):#len(list_of_imgs)):
    imgs    += 1
    img = load_img(list_of_imgs[im])  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (1024, 2048, 3)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 1024, 2048, 3)
    dataset_calib = np.append(dataset_calib, x, axis=0)
print('Calibration data: {}, {}, {}'.format(dataset_calib.shape, dataset_calib.ndim, dataset_calib.dtype))

np.savez('./gen/calib_data.npz', data = dataset_calib[:num_imgs])

del(dataset_calib)

Calibration data: (1, 256, 640, 3), 4, float32


In [21]:
%%writefile input_func.py
import numpy as np

data = np.load('./gen/calib_data.npz')['data']

batch_size=1

def calib_input(iter):

    calib_data = data[iter*batch_size:(iter+1)*batch_size]

    return {'input_1': calib_data}

Writing input_func.py


In [18]:
!vai_q_tensorflow quantize --help

usage: 
    usage: vai_q_tensorflow command [Options]
    
    examples:
      show help       : vai_q_tensorflow --help
      quantize a model: vai_q_tensorflow quantize --input_frozen_graph frozen_graph.pb --input_nodes xxx --output_nodes yyy --input_shapes zzz --input_fn module.calib_input
      inspect a model : vai_q_tensorflow inspect --input_frozen_graph frozen_graph.pb
      dump quantized model : vai_q_tensorflow dump --input_frozen_graph quantize_results/quantize_eval_model.pb --input_fn module.dump_input
  

Xilinx's Quantization Tools Vai_q_tensorflow v1.0.0 Build for Tensorflow
1.12.0

positional arguments:
  {quantize,inspect,dump}
                        Specify a command for vai_q_tensorflow

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  --input_frozen_graph INPUT_FROZEN_GRAPH
                        The path to input frozen graph(.pb) (default: )
  --output_dir OUTPUT_DIR
   

In [22]:
!vai_q_tensorflow quantize \
    --input_frozen_graph ./gen/frozen.pb \
    --input_fn input_func.calib_input \
    --output_dir quantized \
    --input_nodes input_1 \
    --input_shapes ?,256,640,3 \
    --output_nodes conv2d_68/BiasAdd \
    --method 0 \
    --align_concat 2 \
    --weight_bit 8 \
    --activation_bit 8 \
    --skip_check 0 \
    --simulate_dpu 0 \
    --calib_iter 1

INFO: Checking Float Graph...
INFO: Float Graph Check Done.
INFO: Calibrating for 1 iterations...
100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
INFO: Calibration Done.
INFO: Generating Deploy Model...
INFO: Deploy Model Generated.
********************* Quantization Summary *********************      
INFO: Output:       
  quantize_eval_model: quantized/quantize_eval_model.pb       
  deploy_model: quantized/deploy_model.pb


In [23]:
!vai_c_tensorflow --help

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
usage: vai_c_tensorflow [-h] [-f FROZEN_PB] [-a ARCH] [-o OUTPUT_DIR]
                        [-n NET_NAME] [-e OPTIONS] [-q]

optional arguments:
  -h, --help            show this help message and exit
  -f FROZEN_PB, --frozen_pb FROZEN_PB
                        prototxt
  -a ARCH, --arch ARCH  json file
  -o OUTPUT_DIR, --output_dir OUTPUT_DIR
                        output directory
  -n NET_NAME, --net_name NET_NAME
                        prefix-name for the outputs
  -e OPTIONS, --options OPTIONS
                        extra options
  -q, --quant_info      extract quant info


In [26]:
!vai_c_tensorflow \
    --frozen_pb ./quantized/deploy_model.pb \
    --arch /opt/vitis_ai/compiler/arch/dpuv2/Ultra96/Ultra96.json \
    --output_dir ./gen \
    --net_name tf_poly_yolo \
    --quant_info

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
/opt/vitis_ai/compiler/arch/dpuv2/Ultra96/Ultra96.json

Kernel topology "tf_poly_yolo_kernel_graph.jpg" for network "tf_poly_yolo"
kernel list info for network "tf_poly_yolo"
                               Kernel ID : Name
                                       0 : tf_poly_yolo

                             Kernel Name : tf_poly_yolo
--------------------------------------------------------------------------------
                             Kernel Type : DPUKernel
                               Code Size : 0.72MB
                              Param Size : 15.58MB
                           Workload MACs : 17715.69MOPS
                         IO Memory Space : 4.22MB
                              Mean Value : 0, 0, 0, 
                      Total Tensor Count : 71
                Boundary Input Tensor(s)   (H*W*C)
                            input_